In [3]:
import openai

In [18]:
# key_g = ""
# key_i = ""
# openai.api_key = key_g

## Document Loading

In [19]:
from langchain.document_loaders import PyPDFLoader

In [20]:
loader_one = PyPDFLoader("test.pdf")

In [21]:
pages = loader_one.load()

In [22]:
len(pages)

3

In [23]:
page = pages[0]

In [24]:
page.metadata

{'source': 'test.pdf', 'page': 0}

In [25]:
page.page_content[500:1500]

'itization, multiple ways of attacking\nthrough code have emerged. Allowing cybercriminals to access\nconfidential information, impersonate identities, steal money,\naccess databases, among others. Our project approaches the 3 of\n10 vulnerabilities identified by OWASP as the most recognized\nand usual in web applications, to identify and evaluate them at\nthe JavaScript code level. For this, a machine learning model was\nproposed, detecting the frequency of occurrence of a word with\nthe identified vulnerability; resulting in a model with an accuracy\nof 89%. Finally, an extension was implemented in Visual Studio\nCode to read in real time the code that the person is writing in\norder to identify which vulnerabilities it has.\nIndex Terms —OWASP, Front-End Vulnerabilities, Machine\nLearning, Artificial Intelligence.\nI. INTRODUCTION\nFront-End developers do not have enough tools that en-\ncompass the complex diversity of cybersecurity challenges.\nPart of that inadequacy comes from cu

In [1]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

Pay attention that we have a limitation of requests using openai. The next request will cost USD $0.76

In [8]:
url = "https://youtu.be/_kr-XOeZmW4"
loader = GenericLoader(
    YoutubeAudioLoader([url],"."),
    OpenAIWhisperParser())
docs = loader.load()


[youtube] Extracting URL: https://youtu.be/_kr-XOeZmW4
[youtube] _kr-XOeZmW4: Downloading webpage
[youtube] _kr-XOeZmW4: Downloading ios player API JSON
[youtube] _kr-XOeZmW4: Downloading android player API JSON
[youtube] _kr-XOeZmW4: Downloading m3u8 information
[youtube] _kr-XOeZmW4: Downloading MPD manifest
[info] _kr-XOeZmW4: Downloading 1 format(s): 140
[download] ytcracker - enter my world (produced by amplitude problem).m4a has already been downloaded
[download] 100% of    2.98MiB
[ExtractAudio] Not converting audio ytcracker - enter my world (produced by amplitude problem).m4a; file is already in target format m4a
Transcribing part 1!


In [17]:
docs[0].page_content

"BASS BOOSTED MUSIC Look around what do you see, couple handfuls of hackers with their black hat mentality Most of us got records cause we feel compelled to flex All compulsion for these systems that we probably born to wreck My respect to my hackahgotten ex Hacking our election process like a boss bitch This type of terrorism really look terrible Ain't no bloodshed, nobody countin' for some barrels Had a few here, so check a few there Load up all your Twitter bots, I'll teach a crew there They're the marmade, foldin' up the shop origami Diamond, betting on the horses, betting on the jockeys Unh, unh, unh, unh Stick a fork in a book, a process, stick a fork in em Database pushed right to the open Google dork in em, stick a fork in a book, a process, stick a fork in em End of my world, there's no bake as it look There's dark artists doing modesty, no page in the book Can teach you anything, we learn it, no mistake and we cook And we be chopping up the blades like a fleet of Chinooks And

In [ ]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("document")
docs = loader.load()
docs[0].page_content[:200]

## Document Splitting

In [28]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 150,
    length_function=len
)

In [43]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [47]:
text_splitter.split_text(docs[0].page_content)[:20]

['B',
 'ASS',
 ' B',
 'OO',
 'ST',
 'ED',
 ' MUS',
 'IC',
 ' Look',
 ' around',
 ' what',
 ' do',
 ' you',
 ' see',
 ',',
 ' couple',
 ' handful',
 's',
 ' of',
 ' hackers']

In [44]:
docs_splits = text_splitter.split_documents(docs)

In [49]:
text_splitter.split_documents(pages)[:4]

[Document(page_content='C', metadata={'source': 'test.pdf', 'page': 0}),
 Document(page_content='ognitive', metadata={'source': 'test.pdf', 'page': 0}),
 Document(page_content=' Solution', metadata={'source': 'test.pdf', 'page': 0}),
 Document(page_content=' for', metadata={'source': 'test.pdf', 'page': 0})]

In [37]:
print(len(docs))
print(len(docs_splits))

1
1


In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os
import openai
from langchain.document_loaders import PyPDFLoader

In [10]:
loaders = [
    PyPDFLoader("test.pdf"),
    PyPDFLoader("semi-automated.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [13]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [14]:
splits = text_splitter.split_documents(docs)

In [15]:
len(splits)

88

### Embeddings

In [19]:
openai.api_key = key_g

In [24]:
embedding = OpenAIEmbeddings(openai_api_key=key_g)

In [25]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [26]:
import numpy as np

In [27]:
np.dot(embedding1, embedding2)

0.963208056126645

In [28]:
np.dot(embedding2, embedding3)

0.7596001131741936

In [29]:
np.dot(embedding1, embedding3)

0.7709997651294672

## Vectorstores